In [45]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import Dense, Add, Multiply, LayerNormalization, Dropout
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint, EarlyStopping
os.environ["OMP_NUM_THREADS"] = "2"
os.environ["TF_NUM_INTRAOP_THREADS"] = "2"
os.environ["TF_NUM_INTEROP_THREADS"] = "2"

print("CPU threads are limited to 2.")

import warnings

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

CPU threads are limited to 2.


In [46]:
vocab_size = 15000  # Size of vocabulary for hash and path features
embedding_dim = 16  # Embedding dimension
max_path_length = 20  # Maximum length for path sequences

save_model_dir = "F:/models"
best_model_path = f"{save_model_dir}/best_model.keras"
epoch_model_path = f"{save_model_dir}/model_epoch_{'{epoch:02d}'}.keras"

In [47]:
def create_hash_embedding_inputs(hash_features):
    """
    Create embedding layers and input layers for hash-based features.
    """
    hash_inputs = []
    hash_embeddings = []

    for feature in hash_features:
        input_layer = Input(shape=(1,), name=f"{feature}_input")
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1)(input_layer)
        hash_inputs.append(input_layer)
        hash_embeddings.append(Flatten()(embedding_layer))  # Flatten to use with Dense layers later

    return hash_inputs, hash_embeddings

In [48]:
def create_path_embedding_inputs(path_features, use_shared_embedding=True):
    """
    Create embedding layers and input layers for path-based features.
    If `use_shared_embedding` is True, a single embedding layer is shared for all path features.
    """
    path_inputs = []
    path_embeddings = []

    if use_shared_embedding:
        shared_embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_path_length)

        for feature in path_features:
            input_layer = Input(shape=(max_path_length,), name=f"{feature}_input")
            embedding_output = shared_embedding_layer(input_layer)
            path_inputs.append(input_layer)
            path_embeddings.append(Flatten()(embedding_output))  # Flatten to combine with other features

    else:
        for feature in path_features:
            input_layer = Input(shape=(max_path_length,), name=f"{feature}_input")
            embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_path_length)(
                input_layer
            )
            path_inputs.append(input_layer)
            path_embeddings.append(Flatten()(embedding_layer))

    return path_inputs, path_embeddings

In [49]:
def create_textual_embedding_inputs(textual_features, tokenizer_vocab_size):
    """
    Create embedding layers and input layers for textual features.
    Each feature will have its own embedding layer.
    """
    text_inputs = []
    text_embeddings = []

    for feature in textual_features:
        input_layer = Input(shape=(max_path_length,), name=f"{feature}_input")  # Textual features are tokenized
        embedding_layer = Embedding(input_dim=tokenizer_vocab_size, output_dim=embedding_dim, input_length=max_path_length)(
            input_layer
        )
        text_inputs.append(input_layer)
        text_embeddings.append(Flatten()(embedding_layer))  # Flatten for further processing

    return text_inputs, text_embeddings

In [50]:
def create_numerical_inputs(numerical_features):
    """
    Create input layers for numerical (normalized) features.
    """
    numerical_inputs = []
    for feature in numerical_features:
        input_layer = Input(shape=(1,), name=f"{feature}_input")  # Each feature has its own input layer
        numerical_inputs.append(input_layer)

    return numerical_inputs

In [51]:
hash_features = [
    "Level_hash", "Task_hash", "Protocol_hash", "Version2_hash", "SchemaVersion_hash",
    "Signature_hash", "EventType_hash", "StartFunction_hash", "ID_hash",
    "Configuration_hash", "ConfigurationFileHash_hash", "IntegrityLevel_hash",
    "UserID_hash", "Computer_hash", "RuleName_hash", "TerminalSessionId_hash",
    "Version_hash", "User_hash", "ParentUser_hash", "FileVersion_hash",
    "Guid_hash", "ProcessGuid_hash", "ParentProcessGuid_hash", "SourceProcessGUID_hash",
    "TargetProcessGUID_hash", "SourceProcessGuid_hash", "LogonGuid_hash",
    "ThreadID_hash", "LogonId_hash", "SourceHostname_hash", "OriginalFileName_hash",
    "TargetProcessGuid_hash", "DestinationHostname_hash", "SourcePortName_hash",
    "DestinationPortName_hash", "StartAddress_hash", "StartModule_hash", "NewThreadId_hash",
    "GrantedAccess_hash", "QueryName_hash", "QueryResults_hash", "Hashes_hash", 
    "Hash_hash", "Contents_hash", "SourceIp_hash", "SourceIp_prefix_hash", 
    "DestinationIp_hash", "DestinationIp_prefix_hash", "EventID_hash", 
    "ProcessID_hash", "Execution_ProcessID_hash", "ParentProcessId_hash", 
    "TargetProcessId_hash", "EventRecordID_hash", "SourcePort_hash", 
    "DestinationPort_hash", "ProcessId_hash"
]
 # Replace with your hash features
path_features = [
    "Image", "ParentImage", "SourceImage", "TargetImage", "ImageLoaded",
    "CurrentDirectory", "ParentCommandLine", "CommandLine", "CallTrace", 
    "TargetFilename", "TargetObject", "Details", "PipeName", "QueryName"
]

  # Replace with your path features
textual_features = [
    "Description", "Product", "Company"
]
 # Replace with your textual features
numerical_features = [
    "SystemTime", "UtcTime", "CreationUtcTime", "PreviousCreationUtcTime"
]
# Create embedding and input layers
hash_inputs, hash_embeddings = create_hash_embedding_inputs(hash_features)
path_inputs, path_embeddings = create_path_embedding_inputs(path_features, use_shared_embedding=True)
text_inputs, text_embeddings = create_textual_embedding_inputs(textual_features, tokenizer_vocab_size=10000)
numerical_inputs = create_numerical_inputs(numerical_features)

# Combine all embeddings and inputs
all_inputs = hash_inputs + path_inputs + text_inputs + numerical_inputs
all_embeddings = hash_embeddings + path_embeddings + text_embeddings

# Numerical features are already tensors; no embedding layer required
numerical_tensors = numerical_inputs


In [52]:
def gating_mechanism(x, units, dense_layer):
    """Applies a gating mechanism to input x using a pre-defined Dense layer."""
    activation_layer = dense_layer(x)
    return Multiply()([x, activation_layer])


In [53]:
def scaled_dot_product_attention(query, key, value, mask=None):
    """Calculates scaled dot-product attention."""
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    dk = tf.cast(tf.shape(key)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)
    
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, value)
    return output, attention_weights


In [54]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)
    
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, query, key, value, mask=None):
        batch_size = tf.shape(query)[0]
        query = self.split_heads(self.wq(query), batch_size)
        key = self.split_heads(self.wk(key), batch_size)
        value = self.split_heads(self.wv(value), batch_size)
        
        attention, weights = scaled_dot_product_attention(query, key, value, mask)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.num_heads * self.depth))
        output = self.dense(concat_attention)
        return output, weights


In [55]:
class TemporalFusionTransformer(tf.keras.Model):
    def __init__(self, static_input_dim, time_varying_input_dim, output_dim, d_model, num_heads, dropout_rate=0.1):
        super(TemporalFusionTransformer, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads

        # Static embeddings
        self.static_embedding = Dense(d_model)
        self.gating_dense = Dense(d_model, activation="sigmoid")  # Define Dense layer for gating

        # Temporal embeddings
        self.temporal_embedding = Dense(d_model)

        # Multi-head attention layers
        self.multi_head_attention = MultiHeadAttention(d_model, num_heads)

        # Feed-forward layers
        self.ffn = tf.keras.Sequential([
            Dense(d_model, activation="relu"),
            Dense(d_model)
        ])

        # Normalization and dropout
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.dropout = Dropout(dropout_rate)

        # Pooling layer to aggregate time steps
        self.pooling = tf.keras.layers.GlobalAveragePooling1D()

        # Output layer
        self.output_layer = Dense(output_dim, activation="sigmoid")  # Binary classification

    def call(self, static_inputs, temporal_inputs):
        # Static processing
        static_embedded = self.static_embedding(static_inputs)

        # Apply gating mechanism
        gated_static = gating_mechanism(static_embedded, self.d_model, self.gating_dense)

        # Temporal processing
        temporal_embedded = self.temporal_embedding(temporal_inputs)

        # Attention mechanism
        attention_output, _ = self.multi_head_attention(
            query=temporal_embedded, key=temporal_embedded, value=temporal_embedded
        )
        attention_output = self.dropout(attention_output)
        attention_output = self.norm1(attention_output + temporal_embedded)

        # Feed-forward network
        ffn_output = self.ffn(attention_output)
        ffn_output = self.dropout(ffn_output)
        ffn_output = self.norm2(ffn_output + attention_output)

        # Combine static and temporal
        gated_static_expanded = tf.expand_dims(gated_static, axis=1)
        combined = Add()([ffn_output, gated_static_expanded])

        # Aggregate across timesteps (pooling)
        pooled_output = self.pooling(combined)

        # Output
        output = self.output_layer(pooled_output)
        return output


In [56]:
# Step 1: Define TFT with Preprocessed Features
def build_tft_with_embeddings(
    d_model, num_heads, dropout_rate, static_input_dim, temporal_input_dim
):
    """
    Builds a Temporal Fusion Transformer (TFT) model.

    Args:
        d_model: Dimensionality of the model layers.
        num_heads: Number of attention heads in the transformer.
        dropout_rate: Dropout rate for regularization.
        static_input_dim: Dimensionality of static inputs.
        temporal_input_dim: Dimensionality of temporal inputs.

    Returns:
        A compiled TFT model.
    """
    # Input layers for static and temporal features
    static_input_layer = Input(shape=(static_input_dim,), name="static_input")
    temporal_input_layer = Input(shape=(None, temporal_input_dim), name="temporal_input")

    # Instantiate the Temporal Fusion Transformer model
    tft = TemporalFusionTransformer(
        static_input_dim=static_input_dim,
        time_varying_input_dim=temporal_input_dim,
        output_dim=1,  # Binary classification
        d_model=d_model,
        num_heads=num_heads,
        dropout_rate=dropout_rate,
    )

    # Get outputs from the TFT model
    outputs = tft(static_input_layer, temporal_input_layer)

    # Create and compile the Keras model
    model = Model(inputs=[static_input_layer, temporal_input_layer], outputs=outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    return model

# Step 2: Prepare Inputs for Model
def prepare_tft_inputs(hash_embeddings, path_embeddings, text_embeddings, numerical_inputs):
    """
    Prepares inputs for the TFT model.

    Args:
        hash_embeddings: List of tensors from hash feature embeddings.
        path_embeddings: List of tensors from path feature embeddings.
        text_embeddings: List of tensors from textual feature embeddings.
        numerical_inputs: List of tensors for normalized numerical features.

    Returns:
        A dictionary containing static and temporal input tensors.
    """
    # Concatenate static embeddings (hash, path, and text features)
    static_embeddings = Concatenate()(hash_embeddings + path_embeddings + text_embeddings)

    # Concatenate numerical inputs for temporal features
    temporal_inputs = Concatenate()(numerical_inputs)
    
    return static_embeddings, temporal_inputs


# Combine embeddings for static and temporal features
static_inputs, temporal_inputs = prepare_tft_inputs(hash_embeddings, path_embeddings, text_embeddings, numerical_inputs)

# Build the TFT model
tft_model = build_tft_with_embeddings(
    d_model=64,
    num_heads=8,
    dropout_rate=0.1,
    static_input_dim=217,  # Replace with the actual shape from the generator
    temporal_input_dim=4,  # Replace with actual temporal input dimensions
)



# Step 4: View Model Summary
tft_model.summary()



Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ static_input        │ (None, 217)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ temporal_input      │ (None, None, 4)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ temporal_fusion_tr… │ (None, 1)         │     43,713 │ static_input[0][… │
│ (TemporalFusionTra… │                   │            │ temporal_input[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 43,713 (170.75 KB)

 Trainable params: 43,713 (170.75 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
def data_generator_from_folder(
    folder_path, chunk_size, hash_features, path_features, textual_features, numerical_features, tokenizer, vocab_size, split_ratio=0.8, mode="train"
):
    """
    Generator to produce static and temporal inputs along with labels, split dynamically for train/validation,
    with memory optimizations.
    """
    csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

    for file_path in csv_files:
        print(f"Processing file: {file_path}")
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):  # Allow pandas to infer types
            # Shuffle the chunk
            chunk = chunk.sample(frac=1).reset_index(drop=True)
            
            # Split into training and validation
            split_index = int(len(chunk) * split_ratio)
            if mode == "train":
                data_chunk = chunk.iloc[:split_index]
            else:  # mode == "validation"
                data_chunk = chunk.iloc[split_index:]

            # Preprocess static inputs (hash, path, and textual features)
            static_inputs = []
            for feature in hash_features:
                if feature in data_chunk.columns:
                    static_inputs.append(data_chunk[feature].fillna(0).astype("int32").values)
            for feature in path_features:
                if feature in data_chunk.columns:
                    tokenized = tokenizer.texts_to_sequences(data_chunk[feature].fillna("").astype(str))
                    padded = pad_sequences(tokenized, maxlen=10, padding="post", truncating="post")
                    static_inputs.append(padded)
            for feature in textual_features:
                if feature in data_chunk.columns:
                    tokenized = tokenizer.texts_to_sequences(data_chunk[feature].fillna("").astype(str))
                    padded = pad_sequences(tokenized, maxlen=10, padding="post", truncating="post")
                    static_inputs.append(padded)

            # Combine static inputs into a single array
            static_inputs_combined = np.concatenate(
                [np.expand_dims(arr, axis=1) if arr.ndim == 1 else arr for arr in static_inputs],
                axis=1
            ).astype("float32")  # Ensure memory-efficient dtype

            # Preprocess temporal inputs (numerical features)
            temporal_inputs = []
            for feature in numerical_features:
                if feature in data_chunk.columns:
                    temporal_inputs.append(data_chunk[feature].fillna(0).astype("float32").values.reshape(-1, 1))
            temporal_inputs_combined = np.expand_dims(
                np.concatenate(temporal_inputs, axis=1), axis=1
            )  # Add sequence dimension

            

            # Extract and process labels
            if "Label" in data_chunk.columns:
                labels = data_chunk["Label"].values
                labels = np.where(labels == 2, 1, labels).astype("int32")  # Ensure int32 dtype
            else:
                raise KeyError("The 'Label' column is missing from the dataset.")

            # Yield combined inputs and labels
            yield {"static_input": static_inputs_combined, "temporal_input": temporal_inputs_combined}, labels


In [58]:
folder_path = "F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed"  
chunk_size = 10000  # Number of rows to read per chunk

# Define features
hash_features = [
    "Level_hash", "Task_hash", "Protocol_hash", "Version2_hash", "SchemaVersion_hash",
    "Signature_hash", "EventType_hash", "StartFunction_hash", "ID_hash",
    "Configuration_hash", "ConfigurationFileHash_hash", "IntegrityLevel_hash",
    "UserID_hash", "Computer_hash", "RuleName_hash", "TerminalSessionId_hash",
    "Version_hash", "User_hash", "ParentUser_hash", "FileVersion_hash",
    "Guid_hash", "ProcessGuid_hash", "ParentProcessGuid_hash", "SourceProcessGUID_hash",
    "TargetProcessGUID_hash", "SourceProcessGuid_hash", "LogonGuid_hash",
    "ThreadID_hash", "LogonId_hash", "SourceHostname_hash", "OriginalFileName_hash",
    "TargetProcessGuid_hash", "DestinationHostname_hash", "SourcePortName_hash",
    "DestinationPortName_hash", "StartAddress_hash", "StartModule_hash", "NewThreadId_hash",
    "GrantedAccess_hash", "QueryName_hash", "QueryResults_hash", "Hashes_hash", 
    "Hash_hash", "Contents_hash", "SourceIp_hash", "SourceIp_prefix_hash", 
    "DestinationIp_hash", "DestinationIp_prefix_hash", "EventID_hash", 
    "ProcessID_hash", "Execution_ProcessID_hash", "ParentProcessId_hash", 
    "TargetProcessId_hash", "EventRecordID_hash", "SourcePort_hash", 
    "DestinationPort_hash", "ProcessId_hash"
]
 # Replace with your hash features
path_features = [
    "Image", "ParentImage", "SourceImage", "TargetImage", "ImageLoaded",
    "CurrentDirectory", "ParentCommandLine", "CommandLine", "CallTrace", 
    "TargetFilename", "TargetObject", "Details", "PipeName", "QueryName"
]

  # Replace with your path features
textual_features = [
    "Description", "Product", "Company"
]
 # Replace with your textual features
numerical_features = [
    "SystemTime", "UtcTime", "CreationUtcTime", "PreviousCreationUtcTime"
]
  # Replace with your numerical features

# Initialize tokenizer (use the one created earlier)

tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts([])  # Adjust to fit your corpus if required



In [59]:
if not os.path.exists(save_model_dir):
    os.makedirs(save_model_dir)

In [60]:
callbacks = [
    ModelCheckpoint(
        filepath=epoch_model_path,  # Save model at the end of each epoch
        save_best_only=False,
        save_weights_only=False,
        verbose=1,
    ),
    ModelCheckpoint(
        filepath=best_model_path,  # Save the best model
        save_best_only=True,
        save_weights_only=False,
        monitor="val_loss",  # Monitor validation loss
        mode="min",
        verbose=1,
    ),
    EarlyStopping(
        monitor="val_loss",  # Stop training if validation loss doesn't improve
        patience=5,  # Number of epochs to wait
        verbose=1,
        restore_best_weights=True,
    )
]


In [61]:
# Create training generator
train_generator = data_generator_from_folder(
    folder_path="F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed",
    chunk_size=10000,
    hash_features=hash_features,
    path_features=path_features,
    textual_features=textual_features,
    numerical_features=numerical_features,
    tokenizer=tokenizer,
    vocab_size=15000,
    split_ratio=0.8,
    mode="train"
)

# Create validation generator
validation_generator = data_generator_from_folder(
    folder_path="F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed",
    chunk_size=10000,
    hash_features=hash_features,
    path_features=path_features,
    textual_features=textual_features,
    numerical_features=numerical_features,
    tokenizer=tokenizer,
    vocab_size=15000,
    split_ratio=0.8,
    mode="validation"
)


In [62]:
# Training configuration
epochs = 20  # Number of training epochs
steps_per_epoch = 100  # Number of steps per epoch (based on chunk size and dataset size)
validation_steps = 20  # Number of validation steps

In [63]:
# Compile the model
tft_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",  # For binary classification
    metrics=["accuracy"],
)

In [64]:
# Test the generator
gen = data_generator_from_folder(
    folder_path="F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed",
    chunk_size=10000,
    hash_features=hash_features,
    path_features=path_features,
    textual_features=textual_features,
    numerical_features=numerical_features,
    tokenizer=tokenizer,
    vocab_size=15000,
    mode="train"
)

inputs, labels = next(gen)
print("Static input shape:", inputs["static_input"].shape)
print("Temporal input shape:", inputs["temporal_input"].shape)
print("Labels shape:", labels.shape)



Processing file: F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed\LMD-2022 [870K Elements][Labelled].csv
Static input shape: (8000, 217)
Temporal input shape: (8000, 1, 4)
Labels shape: (8000,)


In [65]:
tft_model.fit(
    train_generator,
    steps_per_epoch=532,  # Adjust based on your data size
    validation_data=validation_generator,
    validation_steps=134,  # Adjust based on your data size
    epochs=30,
    callbacks=callbacks,
)


Processing file: F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed\LMD-2022 [870K Elements][Labelled].csv
Epoch 1/30
 86/532 ━━━━━━━━━━━━━━━━━━━━ 4:54 661ms/step - accuracy: 0.8690 - loss: 2301.5073Processing file: F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed\LMD-2023 [1.75M Elements][Labelled]checked.csv
261/532 ━━━━━━━━━━━━━━━━━━━━ 3:37 804ms/step - accuracy: 0.9441 - loss: 952.0029Processing file: F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed\LMD-2023 [1.87M Elements][Labelled]checked.csv
452/532 ━━━━━━━━━━━━━━━━━━━━ 1:07 844ms/step - accuracy: 0.9533 - loss: 1204.2612Processing file: F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed\LMD-2023 [2.3M Elements][Labelled]checked.csv
532/532 ━━━━━━━━━━━━━━━━━━━━ 0s 857ms/step - accuracy: 0.9524 - loss: 1580.7924Processing file: F:/Intrusion

In [66]:
# def calculate_steps(folder_path, chunk_size, split_ratio=0.8):
#     """
#     Calculate total steps for training and validation based on the number of rows in the dataset.

#     Args:
#         folder_path (str): Path to the folder containing CSV files.
#         chunk_size (int): Number of rows per chunk.
#         split_ratio (float): Proportion of data for training (e.g., 0.8 for 80%).

#     Returns:
#         (int, int): Steps for training and validation.
#     """
#     total_rows = 0
#     csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

#     for file_path in csv_files:
#         # Count rows in each file
#         total_rows += sum(1 for _ in open(file_path)) - 1  # Subtract 1 for the header

#     # Calculate steps
#     total_chunks = total_rows // chunk_size
#     train_steps = int(total_chunks * split_ratio)
#     val_steps = total_chunks - train_steps

#     return train_steps, val_steps

# folder_path = "F:/Intrusion detection datasets/Lateral-Movement-Dataset--LMD_Collections/LMD_essential_processed"
# chunk_size = 10000

# # Calculate steps dynamically
# steps_per_epoch, validation_steps = calculate_steps(folder_path, chunk_size, split_ratio=0.8)

# print(f"Steps per epoch (training): {steps_per_epoch}")
# print(f"Validation steps: {validation_steps}")
